# Simple Satellite Image Downloader

## This notebook provides a simple interface for batch downloading imagery to your computer

* Run the first cell and choose an output folder off your choice from the pop-up window (disable pop up blocker, just for a second, I'm sorry)

* Name your site, choose an available satellite (will be updating) and enter in a start and end date

* Using an Ipyleaflet map, draw a rectange of your region of interest.

* Then see how many images are available for the chosen satellite and fire away!



In [1]:
from coast_corr import coast_corr, SDS_tools, SDS_preprocess
import warnings
#warnings.filterwarnings("ignore")
import ipywidgets
import os
import osmnx as ox
from tkinter import filedialog
from tkinter import *
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl, basemaps, basemap_to_tiles
)

In [2]:
outfolder = os.getcwd()+'/Data/'


destination0=ipywidgets.Text(
    value='ROI Name',
    placeholder='Type something',
    description='ROI Name:',
    disabled=False
)
display(destination0)

jpg=ipywidgets.Checkbox(
    value=False,
    description='Check for JPEG files',
    disabled=False,
    indent=False
)

display(jpg)



sat=ipywidgets.Select(
    options=['Sentinel 2', 'Landsat 5', 'Landsat 7', 'Landsat 8','MODIS Aqua','MODIS Terra'],
    value='Sentinel 2',
    # rows=10,
    description='Satellite:',
    disabled=False
)

display(sat)


y0=ipywidgets.Text(
    value='yyyy-mm-dd',
    placeholder='Type something',
    description='Start Date:',
    disabled=False
)

display(y0)

y1=ipywidgets.Text(
    value='yyyy-mm-dd',
    placeholder='Type something',
    description='End Date:',
    disabled=False
)

display(y1)


Text(value='ROI Name', description='ROI Name:', placeholder='Type something')

Checkbox(value=False, description='Check for JPEG files', indent=False)

Select(description='Satellite:', options=('Sentinel 2', 'Landsat 5', 'Landsat 7', 'Landsat 8', 'MODIS Aqua', '…

Text(value='yyyy-mm-dd', description='Start Date:', placeholder='Type something')

Text(value='yyyy-mm-dd', description='End Date:', placeholder='Type something')

## Draw your ROI. For finer resolution satellites draw a smaller area (<~10000 hectares). 

In [3]:

watercolor = basemap_to_tiles(basemaps.Esri.WorldTopoMap, crs='espg:4326')

m = Map(layers=(watercolor, ), center=(42,-128.85), zoom=4)


dc = DrawControl(
                 rectangle={'shapeOptions': {'color': '#0000FF'}, "primary_length_unit": 'kilometers'},
                 )
def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)
m



Map(center=[42, -128.85], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

created
{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#0000FF', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-124.785733, 41.087028], [-124.785733, 41.86351], [-123.810751, 41.86351], [-123.810751, 41.087028], [-124.785733, 41.087028]]]}}
deleted
{'type': 'Feature', 'properties': {'style': {'pane': 'overlayPane', 'attribution': None, 'bubblingMouseEvents': True, 'fill': True, 'smoothFactor': 1, 'noClip': False, 'stroke': True, 'color': '#0000FF', 'weight': 4, 'opacity': 0.5, 'lineCap': 'round', 'lineJoin': 'round', 'dashArray': None, 'dashOffset': None, 'fillColor': None, 'fillOpacity': 0.2, 'fillRule': 'evenodd', 'interactive': True, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-124.785733, 41.087028], [-124.785733, 41.86351], [-123.810751, 41.86351], [-123.810751, 41.087028], [-124.785733, 41.087028]]]}}
created
{'type': 'Featu

## Check to see what's available

In [4]:
polygon= [(dc.last_draw).get("geometry").get("coordinates")[0]]

# it's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = SDS_tools.smallest_rectangle(polygon)

# date range
if sat.value in ['Sentinel 2','Landsat 5','Landsat 7','Landsat 8']:
    dates = [y0.value, y1.value]
if sat.value in ['MODIS Terra','MODIS Aqua']:
    mon=y1.value.split("-")[1]
    day=y1.value.split("-")[2]
    yr=str(int(round(((int(y0.value.split("-")[0])+int(y1.value.split("-")[0]))/2),0)))
    avg_yr=yr+'-'+mon+'-'+day
    avg_yr
    dates1 = [y0.value, avg_yr]
    dates2 = [avg_yr, y1.value]

# satellite missions
if sat.value=='Sentinel 2':
    satellite='S2'
if sat.value=='Landsat 5':
    satellite='L5'
if sat.value=='Landsat 7':
    satellite='L7'
if sat.value=='Landsat 8':
    satellite='L8'
if sat.value=='MODIS Aqua':
    satellite='MYD'
if sat.value=='MODIS Terra':
    satellite='MOD'
    
sat_list = [satellite]

# name of the site
sitename = destination0.value

# directory where the data will be stored
filepath = outfolder
# put all the inputs into a dictionary
if sat.value in ['Sentinel 2','Landsat 5','Landsat 7','Landsat 8']:
    inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

if sat.value in ['MODIS Terra','MODIS Aqua']:
    inputs1 = {'polygon': polygon, 'dates': dates1, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}
    inputs2 = {'polygon': polygon, 'dates': dates2, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

    
    
# before downloading the images, check how many images are available for your inputs
#SDS_download.check_images_available(inputs);
if sat.value in ['MODIS Terra','MODIS Aqua']:
    coast_corr.check_images_available(inputs1)
    coast_corr.check_images_available(inputs2)
    
if sat.value in ['Sentinel 2','Landsat 5','Landsat 7','Landsat 8']:
    coast_corr.check_images_available(inputs)





Images available between 2002-01-01 and 2012-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  MYD: 3458 images
  Total: 3458 images
Images available between 2012-01-01 and 2022-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  MYD: 3653 images
  Total: 3653 images


In [ ]:
import pandas as pd

if sat.value in ['MODIS Terra','MODIS Aqua']:
    metadata1 = coast_corr.retrieve_images_modis(inputs1)
    metadata2 = coast_corr.retrieve_images_modis(inputs2)

if sat.value in ['Sentinel 2','Landsat 5','Landsat 7','Landsat 8']:
    metadata = coast_corr.retrieve_images(inputs)
    
    


if sat.value in ['Sentinel 2','Landsat 5','Landsat 7','Landsat 8']:
    if jpg == 'True':

        metadata = coast_corr.get_metadata(inputs) 

        settings = { 
            # general parameters:
            'cloud_thresh': 0.5,        # threshold on maximum cloud cover
            'output_epsg': 227,        # epsg code of spatial reference system desired for the output   
            # quality control:
            'check_detection': False,    # if True, shows each shoreline detection to the user for validation
            'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
            'save_figure': False,        # if True, saves a figure showing the mapped shoreline for each image
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_swash_area': 3,     # minimum area (in metres^2) for an object to be labelled as swash
            'buffer_size': 1,         # radius (in metres) for buffer around swash pixels considered in the swash detection
            'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
            'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
            # add the inputs defined previously
            'inputs': inputs
        }

        SDS_preprocess.save_jpg(metadata, settings)
        
if sat.value in ['MODIS Terra','MODIS Aqua']:
        if jpg == 'True':

            metadata = coast_corr.get_metadata(inputs1) 

            settings = { 
                # general parameters:
                'cloud_thresh': 0.5,        # threshold on maximum cloud cover
                'output_epsg': 227,        # epsg code of spatial reference system desired for the output   
                # quality control:
                'check_detection': False,    # if True, shows each shoreline detection to the user for validation
                'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
                'save_figure': False,        # if True, saves a figure showing the mapped shoreline for each image
                # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
                'min_swash_area': 3,     # minimum area (in metres^2) for an object to be labelled as swash
                'buffer_size': 1,         # radius (in metres) for buffer around swash pixels considered in the swash detection
                'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
                'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
                'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
                # add the inputs defined previously
                'inputs': inputs
            }

            SDS_preprocess.save_jpg(metadata, settings)

            metadata = coast_corr.get_metadata(inputs2) 

            settings = { 
                # general parameters:
                'cloud_thresh': 0.5,        # threshold on maximum cloud cover
                'output_epsg': 227,        # epsg code of spatial reference system desired for the output   
                # quality control:
                'check_detection': False,    # if True, shows each shoreline detection to the user for validation
                'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
                'save_figure': False,        # if True, saves a figure showing the mapped shoreline for each image
                # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
                'min_swash_area': 3,     # minimum area (in metres^2) for an object to be labelled as swash
                'buffer_size': 1,         # radius (in metres) for buffer around swash pixels considered in the swash detection
                'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
                'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
                'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
                # add the inputs defined previously
                'inputs': inputs
            }

            SDS_preprocess.save_jpg(metadata, settings)


Images available between 2002-01-01 and 2012-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  MYD: 3458 images
  Total: 3458 images

MYD: 3458 images
9%